In [ ]:
# importing libraries
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras import Model,layers

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pzQhYsonvDzrZfiTVd75")
project = rf.workspace("nagesh-pwywk").project("skin-disease-spiyb")
dataset = project.version(1).download("folder")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = "/content/Skin-Disease-1/train"
test_dir="/content/Skin-Disease-1/test"

In [ ]:
BATCH_SIZE=32
# from tensorflow.keras.applications.VGG16 import preprocess_input
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2
                                   )

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = BATCH_SIZE,
                                                    subset='training',
                                                    class_mode = 'categorical',
                                                    target_size = (224,224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  train_datagen.flow_from_directory( train_dir,
                                                          batch_size  = BATCH_SIZE,
                                                          subset='validation',
                                                          class_mode  = 'categorical',
                                                          target_size = (224,224))

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=1,
                                                  shuffle=False,
                                                  class_mode=None)


Found 3269 images belonging to 9 classes.
Found 814 images belonging to 9 classes.
Found 454 images belonging to 9 classes.


In [ ]:
vggModel =VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
for layers in vggModel.layers:
  layers.trainable=False

In [ ]:
x =Flatten()(vggModel.output)
x=tensorflow.keras.layers.Dense(1024,activation="relu")(x)
x=tensorflow.keras.layers.Dense(9,activation="softmax")(x)
vgg_model=  Model(vggModel.input,x)

In [ ]:
from tensorflow.keras.optimizers import Adam,SGD
vgg_model.compile(loss = 'categorical_crossentropy', optimizer =Adam(lr=0.01), metrics = ['accuracy'])

In [ ]:
BATCH_SIZE=32
n_steps=train_generator.samples//BATCH_SIZE
n_val_steps= validation_generator.samples//BATCH_SIZE
# history = model.fit_generator(
#             train_generator,
#             validation_data = validation_generator,
#             steps_per_epoch = 20,
#             epochs = 100,
#             validation_steps =n_val_steps,
#             verbose = 2)
vgg1_history = vgg_model.fit(train_generator, batch_size=BATCH_SIZE,
                            epochs=25,
                            validation_data=validation_generator,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps)

Epoch 1/25
102/102 [==============================] - 78s 681ms/step - loss: 2.5431 - accuracy: 0.4344 - val_loss: 1.1678 - val_accuracy: 0.5750
Epoch 2/25
102/102 [==============================] - 63s 619ms/step - loss: 1.1031 - accuracy: 0.6070 - val_loss: 1.0064 - val_accuracy: 0.6237
Epoch 3/25
102/102 [==============================] - 62s 602ms/step - loss: 0.9059 - accuracy: 0.6753 - val_loss: 0.9385 - val_accuracy: 0.6225
Epoch 4/25
102/102 [==============================] - 63s 619ms/step - loss: 0.8132 - accuracy: 0.7127 - val_loss: 0.9548 - val_accuracy: 0.6725
Epoch 5/25
102/102 [==============================] - 64s 630ms/step - loss: 0.7816 - accuracy: 0.7155 - val_loss: 0.9902 - val_accuracy: 0.6325
Epoch 6/25
102/102 [==============================] - 69s 682ms/step - loss: 0.6621 - accuracy: 0.7553 - val_loss: 0.9196 - val_accuracy: 0.6662
Epoch 7/25
102/102 [==============================] - 68s 663ms/step - loss: 0.6629 - accuracy: 0.7627 - val_loss: 0.9189 - val_ac

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

true_classes = test_generator.classes
class_indices = train_generator.class_indices  # Remove parentheses
class_indices = dict((v, k) for k, v in class_indices.items())
vgg_preds = vgg_model.predict(test_generator)
pred_classes = np.argmax(vgg_preds, axis=1)

acc = accuracy_score(true_classes, pred_classes)
print("Accuracy: {:.2f}%".format(acc * 100))


454/454 [==============================] - 7s 12ms/step
Accuracy: 89.21%


In [ ]:
import numpy as np
vgg_pred_classes = np.argmax(vgg_preds,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score
accuracy =accuracy_score(true_classes,vgg_pred_classes)
print(accuracy*100)

89.20704845814979


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
vgg_model.save("/content/drive/MyDrive/skinvgg.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
model =load_model("/content/drive/MyDrive/skinvgg.h5")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
import numpy as np


NameError: name 'image' is not defined